In [91]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from datetime import datetime as dt
from sklearn.preprocessing import MinMaxScaler


confirmed_cases = pd.read_csv('Dados/time_series_2019-ncov-Confirmed.csv')
confirmed_deaths = pd.read_csv('Dados/time_series_2019-ncov-Deaths.csv')
confirmed_recovered = pd.read_csv('Dados/time_series_2019-ncov-Recovered.csv')

In [92]:
confirmed_recovered

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20
0,NaN,Thailand,15.0000,101.0000,0,0,0,0,2,2,...,33,34,34,35,35,35,35,41,42,42
1,NaN,Japan,36.0000,138.0000,0,0,0,0,1,1,...,101,118,118,118,118,118,144,144,144,150
2,NaN,Singapore,1.2833,103.8333,0,0,0,0,0,0,...,78,96,96,97,105,105,109,114,114,114
3,NaN,Nepal,28.1667,84.2500,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,0,0,0,...,24,26,26,26,35,42,42,49,60,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463,Bermuda,United Kingdom,32.3078,-64.7505,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
464,NaN,Chad,15.4542,18.7322,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
465,NaN,El Salvador,13.7942,-88.8965,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
466,NaN,Fiji,-17.7134,178.0650,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [93]:
confirmed_cases['Province/State']

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
        ...   
463    Bermuda
464        NaN
465        NaN
466        NaN
467        NaN
Name: Province/State, Length: 468, dtype: object

In [94]:
class MLP():
    def __init__(self):
        pass
    def NormalizeData(self,df):
        self.scaler = MinMaxScaler(feature_range=(-1, 1))
        self.normalized = self.scaler.fit_transform(df)
        print(self.normalized)
        
    def Denormalize(self,dfNormalized):
        pass
    
    
    def PrepareData(self,timesteps):
        i = 0
        self.X = []
        self.Y = []
        while i in range(len(self.normalized - timesteps)):
            input_index = i + timesteps
            label_index = input_index + 1
            if(label_index < len(self.normalized)):
                self.X.append(self.normalized[i:input_index,:])
                self.Y.append(self.normalized[input_index:label_index,2])
            i+=1
        self.X = np.array(self.X)
        self.Y = np.array(self.Y)
        print(self.X)
        print(self.Y)
    
    '''
    def Prepare_Data(self,dataset):
        x = dataset.drop(columns=['Total_Deaths'])
        self.X = x.to_numpy()
        y = dataset['Total_Deaths']
        self.Y = y.to_numpy()
        self.Y = self.Y.astype(float)
    '''
    def Build(self,janela,nmr_parametros):
        self.model = keras.Sequential()
        self.model.add(keras.layers.LSTM(216, input_shape=(janela, nmr_parametros), return_sequences=True))
        self.model.add(keras.layers.LSTM(128, input_shape=(janela, nmr_parametros), return_sequences=True))
        self.model.add(keras.layers.LSTM(64, input_shape=(janela, nmr_parametros), return_sequences=False))
        self.model.add(keras.layers.Dense(16, activation="relu", kernel_initializer="uniform"))
        self.model.add(keras.layers.Dense(1, activation="linear"))
                       
    def RMSE(self,y_true,y_pred):
        print(y_true)
        return tf.cast(keras.backend.sqrt(keras.backend.mean(keras.backend.square(y_pred - y_true))),tf.float32)

    def Fit(self):
        self.model.compile(loss=self.RMSE,optimizer=keras.optimizers.Adam(),metrics=['mae',self.RMSE])
        self.history = self.model.fit(x=self.X,y=self.Y,epochs=50,shuffle=False,verbose=True)
    def Predict(self,data):
        result = self.model.predict(data,verbose=True)
        return result
        

In [95]:
class Data():
    def __init__(self):
        pass
    def PreparaData(self,confirmed,deaths,recovered):
        confirmed = confirmed.drop(columns=['Province/State','Country/Region','Lat','Long'])
        deaths = deaths.drop(columns=['Province/State','Country/Region','Lat','Long'])
        recovered = recovered.drop(columns=['Province/State','Country/Region','Lat','Long'])
        timesteps = []
        total_infetados = []
        total_Days = []
        total_Deaths = []
        total_Recovered = []
        self.new_dataset = pd.DataFrame()
        Ground_Zero = dt.strptime('12/31/19','%m/%d/%y')
        for cols in confirmed.columns:
            timesteps.append(cols)
            total_infetados.append(confirmed[cols].sum())
            current_date = dt.strptime(cols,'%m/%d/%y')
            days_Gone = current_date - Ground_Zero
            total_Days.append(int(days_Gone.days))
        for cols in deaths.columns:
            total_Deaths.append(deaths[cols].sum())
        for cols in recovered.columns:
            total_Recovered.append(recovered[cols].sum())
        self.new_dataset['Total_Cases'] = total_infetados
        self.new_dataset['Total_Recovered'] = total_Recovered
        self.new_dataset['Total_Deaths'] = total_Deaths
        self.new_dataset['Days_Gone'] = total_Days
        pd.DataFrame.to_csv(self.new_dataset,'Dados/new_dataset.csv')

In [96]:
data = Data()
data.PreparaData(confirmed_cases,confirmed_deaths,confirmed_recovered)
print(data.new_dataset)

    Total_Cases  Total_Recovered  Total_Deaths  Days_Gone
0           555               28            17         22
1           653               30            18         23
2           941               36            26         24
3          1434               39            42         25
4          2118               52            56         26
5          2927               61            82         27
6          5578              107           131         28
7          6166              126           133         29
8          8234              143           171         30
9          9927              222           213         31
10        12038              284           259         32
11        16787              472           362         33
12        19881              623           426         34
13        23892              852           492         35
14        27635             1124           564         36
15        30817             1487           634         37
16        3439

In [97]:
model = MLP()
model.Build(7,4)
model.NormalizeData(data.new_dataset)
model.PrepareData(7)

[[-1.         -1.         -1.         -1.        ]
 [-0.99919059 -0.99995284 -0.99979695 -0.96491228]
 [-0.99681193 -0.99981138 -0.99817259 -0.92982456]
 [-0.99274013 -0.99974065 -0.99492386 -0.89473684]
 [-0.98709081 -0.99943414 -0.99208122 -0.85964912]
 [-0.98040908 -0.99922194 -0.98680203 -0.8245614 ]
 [-0.95851383 -0.99813736 -0.97685279 -0.78947368]
 [-0.9536574  -0.99768939 -0.9764467  -0.75438596]
 [-0.93657729 -0.99728857 -0.96873096 -0.71929825]
 [-0.92259439 -0.99542593 -0.96020305 -0.68421053]
 [-0.90515913 -0.99396411 -0.95086294 -0.64912281]
 [-0.865936   -0.98953151 -0.92994924 -0.61403509]
 [-0.84038191 -0.98597128 -0.91695431 -0.57894737]
 [-0.80725409 -0.98057199 -0.9035533  -0.54385965]
 [-0.77633975 -0.97415887 -0.88893401 -0.50877193]
 [-0.75005885 -0.96560017 -0.87472081 -0.47368421]
 [-0.72054032 -0.95324547 -0.85746193 -0.43859649]
 [-0.69800085 -0.93898097 -0.83979695 -0.40350877]
 [-0.67297535 -0.92417419 -0.81949239 -0.36842105]
 [-0.65140221 -0.90762266 -0.79

In [98]:
model.Fit()

Tensor("dense_27_target:0", shape=(None, None), dtype=float32)
Tensor("dense_27_target:0", shape=(None, None), dtype=float32)
Train on 50 samples
Epoch 1/50
Tensor("Cast_1:0", shape=(None, 1), dtype=float32)
Tensor("metrics/RMSE/Cast:0", shape=(None, 1), dtype=float32)
Tensor("Cast_1:0", shape=(None, 1), dtype=float32)
Tensor("metrics/RMSE/Cast:0", shape=(None, 1), dtype=float32)
50/50 [==============================] - 5s 90ms/sample - loss: 0.5873 - mae: 0.5537 - RMSE: 0.5341
Epoch 2/50
50/50 [==============================] - 0s 762us/sample - loss: 0.5477 - mae: 0.5148 - RMSE: 0.5000
Epoch 3/50
50/50 [==============================] - 0s 783us/sample - loss: 0.4901 - mae: 0.4576 - RMSE: 0.4504
Epoch 4/50
50/50 [==============================] - 0s 722us/sample - loss: 0.3922 - mae: 0.3588 - RMSE: 0.3669
Epoch 5/50
50/50 [==============================] - 0s 753us/sample - loss: 0.2356 - mae: 0.1926 - RMSE: 0.2361
Epoch 6/50
50/50 [==============================] - 0s 803us/sample -

In [99]:
prediction = model.Predict(model.X[-7:])
value = model.scaler.inverse_transform(prediction)[:, [2]]
print(value)

7/7 [==============================] - 1s 154ms/sample


ValueError: non-broadcastable output operand with shape (7,1) doesn't match the broadcast shape (7,4)

In [81]:
model.X

array([[-1.        , -0.99979695, -0.99817259, -0.99492386, -0.99208122,
        -0.98680203, -0.97685279],
       [-0.99979695, -0.99817259, -0.99492386, -0.99208122, -0.98680203,
        -0.97685279, -0.9764467 ],
       [-0.99817259, -0.99492386, -0.99208122, -0.98680203, -0.97685279,
        -0.9764467 , -0.96873096],
       [-0.99492386, -0.99208122, -0.98680203, -0.97685279, -0.9764467 ,
        -0.96873096, -0.96020305],
       [-0.99208122, -0.98680203, -0.97685279, -0.9764467 , -0.96873096,
        -0.96020305, -0.95086294],
       [-0.98680203, -0.97685279, -0.9764467 , -0.96873096, -0.96020305,
        -0.95086294, -0.92994924],
       [-0.97685279, -0.9764467 , -0.96873096, -0.96020305, -0.95086294,
        -0.92994924, -0.91695431],
       [-0.9764467 , -0.96873096, -0.96020305, -0.95086294, -0.92994924,
        -0.91695431, -0.9035533 ],
       [-0.96873096, -0.96020305, -0.95086294, -0.92994924, -0.91695431,
        -0.9035533 , -0.88893401],
       [-0.96020305, -0.9508